# Widget Events

Trong bài giảng này, chúng ta sẽ thảo luận về các sự kiện widget, chẳng hạn như các lần nhấp vào nút!

## Special events

`Nút` không được sử dụng để đại diện cho một kiểu dữ liệu. Thay vào đó, button widget được sử dụng để xử lý các cú nhấp chuột. Phương thức `on_click` của `Button` có thể được sử dụng để đăng ký một hàm sẽ được gọi khi nút được nhấp. Chuỗi tài liệu của `on_click` có thể được nhìn thấy bên dưới.

In [1]:
import ipywidgets as widgets

print(widgets.Button.on_click.__doc__)

Register a callback to execute when the button is clicked.

        The callback will be called with one argument, the clicked button
        widget instance.

        Parameters
        ----------
        remove: bool (optional)
            Set to true to remove the callback from the list of callbacks.
        


### Example #1 - on_click

Vì các lần nhấp vào nút là không có trạng thái, chúng được truyền từ front-end đến back-end bằng cách sử dụng các thông báo tùy chỉnh. Bằng cách sử dụng phương thức `on_click`, một nút in thông báo khi nó đã được nhấp được hiển thị bên dưới.

In [2]:
from IPython.display import display
button = widgets.Button(description="Click Me!")
display(button)

def on_button_clicked(b):
    print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Button clicked.


### Example #2 - on_submit

widget `Text` cũng có một sự kiện `on_submit` đặc biệt. Sự kiện `on_submit` kích hoạt khi người dùng nhấn <kbd>enter</kbd>.

In [3]:
text = widgets.Text()
display(text)

def handle_submit(sender):
    print(text.value)

text.on_submit(handle_submit)

Text(value='')


d


## Traitlet events
Thuộc tính widget là các đặc tuyến(traitlets) IPython và các đặc tuyến là eventful. Để xử lý các thay đổi, phương thức `observe` của widget có thể được sử dụng để đăng ký một cuộc gọi lại(callback). Chuỗi tài liệu cho `observe` có thể được nhìn thấy bên dưới.

In [4]:
print(widgets.Widget.observe.__doc__)

Setup a handler to be called when a trait changes.

        This is used to setup dynamic notifications of trait changes.

        Parameters
        ----------
        handler : callable
            A callable that is called when a trait changes. Its
            signature should be ``handler(change)``, where ``change`` is a
            dictionary. The change dictionary at least holds a 'type' key.
            * ``type``: the type of notification.
            Other keys may be passed depending on the value of 'type'. In the
            case where type is 'change', we also have the following keys:
            * ``owner`` : the HasTraits instance
            * ``old`` : the old value of the modified trait attribute
            * ``new`` : the new value of the modified trait attribute
            * ``name`` : the name of the modified trait attribute.
        names : list, str, All
            If names is All, the handler will apply to all traits.  If a list
            of str, handler wil

### Signatures
Được đề cập trong docstring, lệnh gọi lại(callback) đã đăng ký phải có chữ ký `trình xử lý (thay đổi)`(`handler(change)`) trong đó `change` là một từ điển chứa thông tin về thay đổi.

Sử dụng phương pháp này, bạn có thể xem ví dụ về cách xuất giá trị của `IntSlider` khi nó bị thay đổi bên dưới.


In [6]:
int_range = widgets.IntSlider()
display(int_range)

def on_value_change(change):
    print(change['new'])

int_range.observe(on_value_change, names='value')

IntSlider(value=0)

# Linking Widgets

Thông thường, bạn có thể chỉ muốn liên kết các thuộc tính widget với nhau. Đồng bộ hóa các thuộc tính có thể được thực hiện theo cách đơn giản hơn là sử dụng các sự kiện đặc điểm trần(bare traitlets events)

## Linking traitlets attributes in the kernel¶

Phương pháp đầu tiên là sử dụng các chức năng `link` và` dlink` từ mô-đun `traitlets`. Điều này chỉ hoạt động nếu chúng ta đang tương tác với một hạt nhân kernel.

In [7]:
import traitlets

In [8]:
# Create Caption
caption = widgets.Label(value = 'The values of slider1 and slider2 are synchronized')

# Create IntSliders
slider1 = widgets.IntSlider(description='Slider 1')
slider2 =  widgets.IntSlider(description='Slider 2')

# Use trailets to link
l = traitlets.link((slider1, 'value'), (slider2, 'value'))

# Display!
display(caption, slider1, slider2)

Label(value='The values of slider1 and slider2 are synchronized')

IntSlider(value=0, description='Slider 1')

IntSlider(value=0, description='Slider 2')

In [9]:
# Create Caption
caption = widgets.Label(value='Changes in source values are reflected in target1')

# Create Sliders
source = widgets.IntSlider(description='Source')
target1 = widgets.IntSlider(description='Target 1')

# Use dlink
dl = traitlets.dlink((source, 'value'), (target1, 'value'))
display(caption, source, target1)

Label(value='Changes in source values are reflected in target1')

IntSlider(value=0, description='Source')

IntSlider(value=0, description='Target 1')

Hàm `traitlets.link` và` traitlets.dlink` trả về đối tượng `Link` hoặc` DLink`. Liên kết có thể bị phá vỡ bằng cách gọi phương thức `unlink(hủy liên kết)`.

In [ ]:
# May get an error depending on order of cells being run!
l.unlink()
dl.unlink()

### Registering callbacks to trait changes in the kernel-Đăng ký callbacks để xác định các thay đổi trong kernel

Vì các thuộc tính của các widget bên Python là các đặc tuyến(traitlets), bạn có thể đăng ký các trình xử lý cho các sự kiện thay đổi bất cứ khi nào mô hình nhận được các bản cập nhật từ front-end.

Trình xử lý được truyền để quan sát sẽ được gọi với một đối số thay đổi. Đối tượng thay đổi giữ ít nhất một key `type` và một key `name`, tương ứng với loại thông báo và tên của thuộc tính đã kích hoạt thông báo.

Các khóa khác có thể được truyền vào tùy thuộc vào giá trị của `type`. Trong trường hợp type là `change(thay đổi)`, chúng ta cũng có các key sau:
* `owner` : the HasTraits instance
* `old` : the old value of the modified trait attribute
* `new` : the new value of the modified trait attribute
* `name` : the name of the modified trait attribute.


In [10]:
caption = widgets.Label(value='The values of range1 and range2 are synchronized')
slider = widgets.IntSlider(min=-5, max=5, value=1, description='Slider')

def handle_slider_change(change):
    caption.value = 'The slider value is ' + (
        'negative' if change.new < 0 else 'nonnegative'
    )

slider.observe(handle_slider_change, names='value')

display(caption, slider)

Label(value='The values of range1 and range2 are synchronized')

IntSlider(value=1, description='Slider', max=5, min=-5)

## Linking widgets attributes from the client side

Khi đồng bộ hóa các thuộc tính của đặc điểm, bạn có thể gặp phải độ trễ do độ trễ do vòng lặp đến phía máy chủ. Bạn cũng có thể liên kết trực tiếp các thuộc tính tiện ích con trong trình duyệt bằng cách sử dụng các widget liên kết, theo kiểu một chiều hoặc hai chiều.

Các liên kết Javascript vẫn tồn tại khi nhúng các widget vào các trang web html mà không có kernel.

In [11]:
# NO LAG VERSION
caption = widgets.Label(value = 'The values of range1 and range2 are synchronized')

range1 = widgets.IntSlider(description='Range 1')
range2 = widgets.IntSlider(description='Range 2')

l = widgets.jslink((range1, 'value'), (range2, 'value'))
display(caption, range1, range2)

Label(value='The values of range1 and range2 are synchronized')

IntSlider(value=0, description='Range 1')

IntSlider(value=0, description='Range 2')

In [12]:
# NO LAG VERSION
caption = widgets.Label(value = 'Changes in source_range values are reflected in target_range')

source_range = widgets.IntSlider(description='Source range')
target_range = widgets.IntSlider(description='Target range')

dl = widgets.jsdlink((source_range, 'value'), (target_range, 'value'))
display(caption, source_range, target_range)

Label(value='Changes in source_range values are reflected in target_range')

IntSlider(value=0, description='Source range')

IntSlider(value=0, description='Target range')

Hàm `widgets.jslink` trả về một widget `Link(Liên kết)`. Liên kết có thể bị phá vỡ bằng cách gọi phương thức `unlink`.

In [13]:
l.unlink()
dl.unlink()

### The difference between linking in the kernel and linking in the client


To see the difference between the two, go to the [ipywidgets documentation](http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html) and try out the sliders near the bottom. The ones linked in the kernel with `link` and `dlink` are no longer linked, but the ones linked in the browser with `jslink` and `jsdlink` are still linked.

Liên kết trong kernel có nghĩa là liên kết thông qua python. Nếu hai thanh trượt được liên kết trong kernel, khi một thanh trượt được thay đổi, trình duyệt sẽ gửi thông báo đến kernel (trong trường hợp này là python) cập nhật thanh trượt đã thay đổi, widget link trong kernel sau đó sẽ truyền thay đổi đến đối tượng thanh trượt khác trong kernel, và sau đó đối tượng kernel của thanh trượt khác sẽ gửi một thông báo đến trình duyệt để cập nhật các view của thanh trượt khác trong trình duyệt. Nếu kernel không chạy (như trong một trang web tĩnh), thì các điều khiển sẽ không được liên kết.

Liên kết bằng jslink (tức là ở phía trình duyệt) có nghĩa là tạo liên kết trong Javascript. Khi một thanh trượt được thay đổi, Javascript đang chạy trong trình duyệt sẽ thay đổi giá trị của thanh trượt kia trong trình duyệt mà không cần giao tiếp với kernel. Nếu các thanh trượt được gắn với các đối tượng kernel, mỗi thanh trượt sẽ cập nhật các đối tượng phía kernel của chúng một cách độc lập.

Để xem sự khác biệt giữa hai, hãy truy cập [ipywidgets documentation](http://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html) và thử các thanh trượt ở gần cuối. Những cái được liên kết trong kernel với `link` và` dlink` không còn được liên kết nữa, nhưng những cái được liên kết trong trình duyệt với `jslink` và `jsdlink` vẫn được liên kết.



## Continuous updates

Một số widget cung cấp sự lựa chọn với thuộc tính `continuous_update` của chúng giữa việc liên tục cập nhật giá trị hoặc chỉ cập nhật giá trị khi người dùng gửi giá trị (ví dụ: bằng cách nhấn Enter hoặc điều hướng khỏi điều khiển). Trong ví dụ tiếp theo, chúng ta thấy các điều khiển "Delayed" chỉ truyền giá trị của chúng sau khi người dùng kết thúc việc kéo thanh trượt hoặc gửi hộp văn bản. Các điều khiển "Continous" liên tục truyền các giá trị của chúng khi chúng được thay đổi. Hãy thử nhập một số có hai chữ số vào mỗi hộp văn bản hoặc kéo từng thanh trượt để xem sự khác biệt.


In [14]:
import traitlets
a = widgets.IntSlider(description="Delayed", continuous_update=False)
b = widgets.IntText(description="Delayed", continuous_update=False)
c = widgets.IntSlider(description="Continuous", continuous_update=True)
d = widgets.IntText(description="Continuous", continuous_update=True)

traitlets.link((a, 'value'), (b, 'value'))
traitlets.link((a, 'value'), (c, 'value'))
traitlets.link((a, 'value'), (d, 'value'))
widgets.VBox([a,b,c,d])

Các điều khiển Sliders, `Text`, và `Textarea` được mặc định thành `continue_update = True`. `IntText` và các hộp văn bản khác để nhập số nguyên hoặc số thực được mặc định thành `continuous_update=False` (vì thường bạn sẽ muốn nhập toàn bộ một số trước khi gửi giá trị bằng cách nhấn enter hoặc điều hướng ra khỏi hộp).

# Conclusion
You should now feel comfortable linking Widget events!